In [1]:
import utils

import utils
print(dir(utils))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'answer_user_msg', 'categories_file', 'client', 'create_categories', 'create_products', 'defaultdict', 'delimiter', 'find_category_and_product', 'find_category_and_product_only', 'generate_output_string', 'get_categories', 'get_completion_from_messages', 'get_mentioned_product_info', 'get_product_by_name', 'get_product_list', 'get_products', 'get_products_and_category', 'get_products_by_category', 'get_products_from_query', 'json', 'openai', 'products_file', 'read_string_to_list', 'step_2_system_message', 'step_2_system_message_content', 'step_4_system_message', 'step_4_system_message_content', 'step_6_system_message', 'step_6_system_message_content']


In [3]:
import os
import openai
import sys
sys.path.append('../..')
import utils

import panel as pn  # GUI
pn.extension()

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [5]:
client = openai.OpenAI()
def get_completion_from_messages(messages, model="gpt-3.5-turbo",temperature=0,max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message.content

In [7]:
def process_user_message(user_input, all_messages, debug=True):
    delimiter = "```"
    
    # Step 1: Check input to see if it flags the Moderation API or is a prompt injection
    response = openai.moderations.create(input=user_input)
    moderation_output = response.results[0]

    if moderation_output.flagged:
        print("Step 1: Input flagged by Moderation API.")
        return "Sorry, we cannot process this request."

    if debug: print("Step 1: Input passed moderation check.")
    
    category_and_product_response = utils.find_category_and_product_only(user_input, utils.get_products_and_category())
    #print(print(category_and_product_response)
    # Step 2: Extract the list of products
    category_and_product_list = utils.read_string_to_list(category_and_product_response)
    #print(category_and_product_list)

    if debug: print("Step 2: Extracted list of products.")

    # Step 3: If products are found, look them up
    product_information = utils.generate_output_string(category_and_product_list)
    if debug: print("Step 3: Looked up product information.")

    # Step 4: Answer the user question
    system_message = f"""
    You are a customer service assistant for a large electronic store. \
    Respond in a friendly and helpful tone, with concise answers. \
    Make sure to ask the user relevant follow-up questions.
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
        {'role': 'assistant', 'content': f"Relevant product information:\n{product_information}"}
    ]

    final_response = get_completion_from_messages(all_messages + messages)
    if debug:print("Step 4: Generated response to user question.")
    all_messages = all_messages + messages[1:]

    # Step 5: Put the answer through the Moderation API
    response = openai.moderations.create(input=final_response)
    moderation_output = response.results[0]

    if moderation_output.flagged:
        if debug: print("Step 5: Response flagged by Moderation API.")
        return "Sorry, we cannot provide this information."

    if debug: print("Step 5: Response passed moderation check.")

    # Step 6: Ask the model if the response answers the initial user query well
    user_message = f"""
    Customer message: {delimiter}{user_input}{delimiter}
    Agent response: {delimiter}{final_response}{delimiter}

    Does the response sufficiently answer the question?
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]
    evaluation_response = get_completion_from_messages(messages)
    if debug: print("Step 6: Model evaluated the response.")

    # Step 7: If yes, use this answer; if not, say that you will connect the user to a human
    if "Y" in evaluation_response:  # Using "in" instead of "==" to be safer for model output variation (e.g., "Y." or "Yes")
        if debug: print("Step 7: Model approved the response.")
        return final_response, all_messages
    else:
        if debug: print("Step 7: Model disapproved the response.")
        neg_str = "I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance."
        return neg_str, all_messages

user_input = "tell me about the smartx pro phone and the fotosnap camera, the dslr one. Also what tell me about your tvs"
response,_ = process_user_message(user_input,[])
print(response)

Step 1: Input passed moderation check.
Error: Invalid JSON string
Step 2: Extracted list of products.
Step 3: Looked up product information.
Step 4: Generated response to user question.
Step 5: Response passed moderation check.
Step 6: Model evaluated the response.
Step 7: Model approved the response.
The SmartX Pro phone is a high-performance smartphone with a powerful processor, advanced camera features, and a sleek design. It offers a smooth user experience and great connectivity options.

The FotoSnap camera is a DSLR camera known for its professional-grade image quality, interchangeable lenses, and manual controls. It's perfect for photography enthusiasts and professionals looking to capture stunning photos.

As for our TVs, we offer a wide range of options including LED, OLED, and QLED TVs in various sizes and resolutions. Our TVs are known for their vibrant colors, sharp image quality, and smart features like built-in streaming apps and voice control.

Do you have any specific q

In [9]:
def collect_messages(debug=False):
    user_input = inp.value_input
    if debug: print(f"User Input = {user_input}")
    if user_input == "":
        return
    inp.value = ''
    global context
    #response, context = process_user_message(user_input, context, utils.get_products_and_category(),debug=True)
    response, context = process_user_message(user_input, context, debug=False)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(user_input, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

In [13]:
panels = [] # collect display 

context = [ {'role':'system', 'content':"You are Service Assistant"} ]  

inp = pn.widgets.TextInput( placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Service Assistant")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Traceback (most recent call last):
 File "C:\Users\nairvis\AppData\Local\anaconda3\Lib\site-packages\pyviz_comms\__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "C:\Users\nairvis\AppData\Local\anaconda3\Lib\site-packages\panel\viewable.py", line 472, in _on_msg
 patch.apply_to_document(doc, comm.id if comm else None)
 File "C:\Users\nairvis\AppData\Local\anaconda3\Lib\site-packages\bokeh\protocol\messages\patch_doc.py", line 104, in apply_to_document
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.payload, setter=setter))
 File "C:\Users\nairvis\AppData\Local\anaconda3\Lib\site-packages\bokeh\document\callbacks.py", line 443, in invoke_with_curdoc
 return f()
 ^^^
 File "C:\Users\nairvis\AppData\Local\anaconda3\Lib\site-packages\bokeh\protocol\messages\patch_doc.py", line 104, in <lambda>
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.payload, setter=setter))
 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\nairvis\AppData\Local\anaconda3\Lib\site-packages\bokeh\document\document.py", line 391, in apply_json_patch
 DocumentPatchedEvent.handle_event(self, event, setter)
 File "C:\Users\nairvis\AppData\Local\anaconda3\Lib\site-packages\bokeh\document\events.py", line 245, in handle_event
 event_cls._handle_event(doc, event)
 File "C:\Users\nairvis\AppData\Local\anaconda3\Lib\site-packages\bokeh\document\events.py", line 280, in _handle_event
 cb(event.msg_data)
 File "C:\Users\nairvis\AppData\Local\anaconda3\Lib\site-packages\bokeh\document\callbacks.py", line 390, in trigger_event
 model._trigger_event(event)
 File "C:\Users\nairvis\AppData\Local\anaconda3\Lib\site-packages\bokeh\util\callback_manager.py", line 113, in _trigger_event
 self.document.callbacks.notify_event(cast(Model, self), event, invoke)
 File "C:\Users\nairvis\AppData\Local\anaconda3\Lib\site-packages\bokeh\document\callbacks.py", line 260, in notify_event
 invoke_with_curdoc(doc, callback_invoker)
 File "C:\Users\nairvis\AppData\Local\anaconda3\Lib\site-packages\bokeh\document\callbacks.py", line 443, in invoke_with_curdoc
 return f()
 ^^^
 File "C:\Users\nairvis\AppData\Local\anaconda3\Lib\site-packages\bokeh\util\callback_manager.py", line 109, in invoke
 cast(EventCallbackWithEvent, callback)(event)
 File "C:\Users\nairvis\AppData\Local\anaconda3\Lib\site-packages\panel\reactive.py", line 484, in _comm_event
 state._handle_exception(e)
 File "C:\Users\nairvis\AppData\Local\anaconda3\Lib\site-packages\panel\io\state.py", line 458, in _handle_exception
 raise exception
 File "C:\Users\nairvis\AppData\Local\anaconda3\Lib\site-packages\panel\reactive.py", line 482, in _comm_event
 self._process_bokeh_event(doc, event)
 File "C:\Users\nairvis\AppData\Local\anaconda3\Lib\site-packages\panel\reactive.py", line 419, in _process_bokeh_event
 self._process_event(event)
 File "C:\Users\nairvis\AppData\Local\anaconda3\Lib\site-packages\panel\widgets\button.py", line 241, in _process_event
 self.param.trigger('value')
 File "C:\Users\nairvis\AppData\Local\anaconda3\Lib\site-packages\param\parameterized.py", line 2473, in trigger
 self_.update(dict(params, **triggers))
 File "C:\Users\nairvis\AppData\Local\anaconda3\Lib\site-packages\param\parameterized.py", line 2318, in update
 restore = dict(self_._update(arg, **kwargs))
 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\nairvis\AppData\Local\anaconda3\Lib\site-packages\param\parameterized.py", line 2351, in _update
 self_._batch_call_watchers()
 File "C:\Users\nairvis\AppData\Local\anaconda3\Lib\site-packages\param\parameterized.py", line 2545, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "C:\Users\nairvis\AppData\Local\anaconda3\Lib\site-packages\param\parameterized.py", line 2505, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "C:\Users\nairvis\AppData\Local\anaconda3\Lib\site-packages\panel\param.py", line 882, in _replace_pane
 new_object = self.eval(self.object)
 ^^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\nairvis\AppData\Local\ana

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Service Assistant')
    [2] ParamFunction(function, _pane=Str, defer_load=False, height=300, loading_indicator=True)